## Example annotating mouse and human peaks to genes

This notebook provides key examples to get started:
    
    1) Annotating peaks from bed files to genes (mouse)
    3) Annotating DNA methylation data to genes (human)

In [1]:
import pandas as pd
from scie2g import Bed, Csv
from sciutil import SciUtil
import igv

data_dir = '../tests/data/'
bed_file = f'{data_dir}test_H3K27ac.bed'
mm10_annot_file = f'{data_dir}mmusculus_gene_ensembl-GRCm38.p6.csv'
u = SciUtil()


## Inspect files

Just to give you an idea of what the files as input are we print out the head of the *annotation* file and the *bed* file.


In [2]:
u.dp(['MM10 Bed file header:'])
print(pd.read_csv(bed_file, '\t'))

u.dp(['MM10 annotation file header:'])
print(pd.read_csv(mm10_annot_file))


--------------------------------------------------------------------------------
                             MM10 Bed file header:	                             
--------------------------------------------------------------------------------
    chr1   13139105   13374083  Peak_147937  12  .  2.20467  3.65974  2.03542  \
0   chr4  118442415  118457513  Peak_126956  13  .  2.16107  4.38555  2.71094   
1  chr15  102326116  102331873   Peak_43115  20  .  3.35950  9.21174  7.28157   
2   chrX   73716597   73738534   Peak_76892  15  .  2.65320  6.01850  4.22238   

   131     Ncoa2  
0    120     Mpl  
1  147     Pfdn5  
2  122     Abcd1  
--------------------------------------------------------------------------------
                         MM10 annotation file header:	                          
--------------------------------------------------------------------------------
          ensembl_gene_id external_gene_name chromosome_name  start_position  \
0      ENSMUSG00000102693      49

## Run bed file annotation

Now we can annotate the bed file to the genes in the annotation file (using the default parameters).

In [3]:
bed = Bed(bed_file)
# Add the gene annot
bed.set_annotation_from_file(mm10_annot_file)
# Now we can run the assign values
bed.assign_locations_to_genes()
bed.save_loc_to_csv(f'test_bed_h3k27ac_output.csv')


4it [00:00, 57.50it/s]

--------------------------------------------------------------------------------
We have added chr prefix to the ensembl chrs.
file: ../tests/data/test_H3K27ac.bed	
--------------------------------------------------------------------------------


## Inspect the output of the annotation

Look at the previous annotation to check the annotations are correct.  

You'll notice that many of the peaks have been annotated to multiple genes, in particular, since the annotation file has ensembl IDs, we get multiple assignments to a single gene name.

In [4]:
u.dp(['Parsed bed and annotated it:'])
print(pd.read_csv('test_bed_h3k27ac_output.csv'))


--------------------------------------------------------------------------------
                         Parsed bed and annotated it:	                          
--------------------------------------------------------------------------------
    peak_idx  gene_idx    chr      start        end  peak_value        8  \
0          0       157   chr1   13139105   13374083     2.20467  2.03542   
1          0       158   chr1   13139105   13374083     2.20467  2.03542   
2          0       159   chr1   13139105   13374083     2.20467  2.03542   
3          0       160   chr1   13139105   13374083     2.20467  2.03542   
4          0       161   chr1   13139105   13374083     2.20467  2.03542   
5          0       162   chr1   13139105   13374083     2.20467  2.03542   
6          0       163   chr1   13139105   13374083     2.20467  2.03542   
7          0       164   chr1   13139105   13374083     2.20467  2.03542   
8          0       165   chr1   13139105   13374083     2.20467  2.03542 

## Override the default parameters

Here we just show how some of the parameters can be overrided.

Here rather than using the peak signal, we choose the peak score (maybe we want it for visualisation purposes). We also add the "Peak" (feature 9, to our output file). We do this by changing: `peak_value=4` (see the features below for the narrow peak format).  

In order to just add extra information, we do this by adding to the `hdridx` and set it to be:
`hdridx='"0","1","2","3","4","8","9"'`. Basically this means: in the output you'll have: `chrom, chromStart,chromEnd,name,score,signalValue,qvalue`.  

Other things we change are the `buffer_before_tss` (the buffer before the TSS in which we want to annotate the peak to the gene) and `buffer_after_tss` (i.e. the amount downstream of the gene end - soz this is poorly named).  


We take the information from the genome browser FAQ: https://genome.ucsc.edu/FAQ/FAQformat.html#format12


0. chrom - Name of the chromosome (or contig, scaffold, etc.).
1. chromStart - The starting position of the feature in the chromosome or scaffold. The first base in a chromosome is numbered 0.
2. chromEnd - The ending position of the feature in the chromosome or scaffold. The chromEnd base is not included in the display of the feature. For example, the first 100 bases of a chromosome are defined as chromStart=0, chromEnd=100, and span the bases numbered 0-99.
3. name - Name given to a region (preferably unique). Use "." if no name is assigned.
4. score - Indicates how dark the peak will be displayed in the browser (0-1000). If all scores were "'0"' when the data were submitted to the DCC, the DCC assigned scores 1-1000 based on signal value. Ideally the average signalValue per base spread is between 100-1000.
5. strand - +/- to denote strand or orientation (whenever applicable). Use "." if no orientation is assigned.
6. signalValue - Measurement of overall (usually, average) enrichment for the region.
7. pValue - Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.
8. qValue - Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.
9. peak - Point-source called for this peak; 0-based offset from chromStart. Use -1 if no point-source called.



In [5]:
bed = Bed(bed_file, overlap_method='overlaps', buffer_after_tss=1000,
                  buffer_before_tss=1000, buffer_gene_overlap=500,
                  gene_start=3, gene_end=4, gene_chr=2,
                  gene_direction=5, gene_name=0,
                  chr_idx=0, start_idx=1,
                  end_idx=2, peak_value=6, header_extra='"7"')
# Add the gene annot
bed.set_annotation_from_file(mm10_annot_file)
# Now we can run the assign values
bed.assign_locations_to_genes()
bed.save_loc_to_csv(f'bed_h3k27ac_output_overlaps.csv')


4it [00:00, 64.94it/s]

--------------------------------------------------------------------------------
We have added chr prefix to the ensembl chrs.
file: ../tests/data/test_H3K27ac.bed	
--------------------------------------------------------------------------------


## Example with CSV file

With the CSV file, we allow for the header to be used (since unlike the bed files we read the whole file into memory). The good thing about the beds is that we only read one line at a time, so it will be able to handle much larger files.

In [6]:
csv_file = f'{data_dir}test_methyl_overlaps.csv'
hg38_annot_file = f'{data_dir}hsapiens_gene_ensembl-GRCh38.p13.csv'

# Print out the files again like we did above
u.dp(['Human CSV file header:'])
print(pd.read_csv(csv_file))

u.dp(['Human annotation file header:'])
print(pd.read_csv(hg38_annot_file))


--------------------------------------------------------------------------------
                            Human CSV file header:	                             
--------------------------------------------------------------------------------
   Unnamed: 0  chr     start       end strand  pvalue  qvalue  meth.diff  \
0           1    7  27070640  27072751      *  0.0005   0.005       -1.0   
1           2    7  27082673  27109403      *  0.0005   0.005       -1.0   
2           3    7  27095203  27096872      *  0.0005   0.005       -1.0   
3           4    7  27095403  27096072      *  0.0005   0.005       -1.0   
4           5    7  27095403  27095404      *  0.0005   0.005       -1.0   

                              description                 genes  
0                           Overlaps None                   NaN  
1  Overlaps Hoxa1 and HotairM1 in TSS ALL  HOTAIRM1,HOXA1,HOXA2  
2    Overlaps Hoxa1 and HotairM1 in TSS A        HOTAIRM1,HOXA1  
3    Overlaps Hoxa1 and HotairM1 in 

## Annotate genes to the CSV file 

Here we show an example of annotating to a csv file. We also show how we can convert a CSV to a bed file both before and after the annotation. This allows us to be able to view the results of the CSV file.

1. Convert the CSV input file to a bed so we can view it in IGV
2. Annotate the features to genes
3. Save csv to a bed file

In [7]:
csv_file = f'{data_dir}test_methyl_overlaps.csv'

f = Csv(csv_file, chr_str='chr', start='start', end='end', value='meth.diff', 
        header_extra=['pvalue', 'qvalue', 'description', 'genes'])

# Convert to bed: 
"""
output filename = test_methyl_overlaps_input
track name = MethylOutputOverlaps_input
value = qvalue
name of peak = genes (a column in the csv file)
"""
f.convert_to_bed(pd.read_csv(csv_file), f'test_methyl_overlaps_input.bed', 'Methyl_input', 
                 'qvalue', 'genes')
f.set_annotation_from_file(hg38_annot_file)

# Now we can run the assign values
f.assign_locations_to_genes()

# Save the output to a csv file
f.save_loc_to_csv(f'test_methyl_overlaps_output.csv')

# We now save the annotated peaks to another bed file so we can inspect the differences.
f.convert_to_bed(f.loc_df, f'test_methyl_overlaps_output.bed', 'MethylOutputOverlaps', 'qvalue', 'external_gene_name')

100%|██████████| 5/5 [00:00<00:00, 252.78it/s]

--------------------------------------------------------------------------------
We have added chr prefix to the ensembl chrs.
file: ../tests/data/test_methyl_overlaps.csv	
--------------------------------------------------------------------------------


## Print out the bed results and the 

If you want to use this section you need to install IGV's browser capabilities in your virtual environment.

You do this the following way (as per documentation at: https://github.com/igvteam/igv-jupyter
```
# To install to configuration in your home directory
jupyter serverextension enable --py igv
jupyter nbextension install --py igv
jupyter nbextension enable --py igv


# If using a virtual environment
jupyter serverextension enable --py igv --sys-prefix
jupyter nbextension install --py igv --sys-prefix
jupyter nbextension enable --py igv --sys-prefix

```

Note if you haven't done the above you will get no visualisation!

In [9]:
import igv

b = igv.Browser({"genome": "hg38"})
b.show()

# Load input track (in purple)
b.load_track(
    {
        "name": "Local Bed",
        "url": "files/test_methyl_overlaps_input.bed",
        "format": "bed",
        "type": "annotation",
        "sourceType": "file",
        "indexed": False,
        "displayMode": "EXPANDED",
        "color": "#8258A1"
    })

# Load output track (in green)
b.load_track(
    {
        "name": "Local Bed",
        "url": "files/test_methyl_overlaps_output.bed",
        "format": "bed",
        "type": "annotation",
        "sourceType": "file",
        "indexed": False,
        "displayMode": "EXPANDED",
        "color": "#8DD9B3"
    })

b.search('HOXA2')
b.get_svg()

b.display_svg()

'Awaiting SVG - try again in a few seconds'